<a href="https://colab.research.google.com/github/hrishikesht7/Ml--small-projects/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.preprocessing import StandardScaler , LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
import joblib


In [ ]:
df = pd.read_csv("loan_approval_dataset Data1.csv")
df.head()


,loan_id,Familt members,education,Turnover,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,0,96000000,29900000,12,778,2400000,17600000,22700000,8000000,1
1,2,0,0,41000000,12200000,8,417,2700000,2200000,8800000,3300000,0
2,3,3,1,91000000,29700000,20,506,7100000,4500000,33300000,12800000,0
3,4,3,1,82000000,30700000,8,467,18200000,3300000,23300000,7900000,0
4,5,5,0,98000000,24200000,20,382,12400000,8200000,29400000,5000000,0


In [ ]:
leabel_encoder = {}
for column in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    leabel_encoder[column]=le

In [ ]:
print(df.columns)


Index(['loan_id', 'Familt members', ' education', 'Turnover', ' loan_amount',
       ' loan_term', ' cibil_score', ' residential_assets_value',
       ' commercial_assets_value', ' luxury_assets_value', ' bank_asset_value',
       ' loan_status'],
      dtype='object')


In [ ]:
X = df.drop(columns=[' loan_status'])
y = df[' loan_status']

In [ ]:
##split the data
X_train , X_test , y_train,y_test = train_test_split(X,y,test_size = 0.2 , random_state = 42,stratify = y)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
## Hyperparamerts Tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_leaf': [1,2,4,6,10],
    'min_samples_split': [2, 5, 10]
}

In [ ]:
## GridsearchCV find best combination of hyperparamerts
grid = GridSearchCV(RandomForestClassifier(random_state=42),param_grid ,cv = 5 , scoring = 'accuracy', n_jobs = -1, verbose = 2)
grid.fit(X_train , y_train)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15],
                         'min_samples_leaf': [1, 2, 4, 6, 10],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=2)

In [ ]:
best_params = grid.best_params_
print("Best parameter:",best_params)


Best parameter: {'max_depth': 15, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [ ]:
model  = RandomForestClassifier(**best_params)
model.fit(X_train , y_train)

RandomForestClassifier(max_depth=15, n_estimators=300)

In [ ]:
Y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test , Y_pred)
print("Accuracy:",accuracy)
print("Classification Report:\n",classification_report(y_test,Y_pred))
print("Confusion Matrix:\n",confusion_matrix(y_test,Y_pred))

Accuracy: 0.9812646370023419
Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.97      0.97       323
           1       0.98      0.99      0.99       531

    accuracy                           0.98       854
   macro avg       0.98      0.98      0.98       854
weighted avg       0.98      0.98      0.98       854

Confusion Matrix:
 [[312  11]
 [  5 526]]


In [ ]:
joblib.dump((model ), 'loan_approval_model.pkl')
joblib.dump(( scaler ), 'loan_approval_model.pkl')
joblib.dump(( leabel_encoder), 'loan_approval_model.pkl')

['loan_approval_model.pkl']